<center><h1>Anne_Sai Venkata Naga Saketh_HW3</h1></center>
<br>
<br>

Name: Anne Sai Venkata Naga Saketh
<br>
Github Username: sakethanne
<br>
USC ID: 3725520208

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [1]:
#all the import statements that shall be required during this assignment are mentioned here in this cell
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import random
import os
import re
import csv
from sklearn.utils import resample

# Package to ignore the Jupyter warnings
import warnings
warnings.filterwarnings('ignore')

#initializing the sniffer object to get the delimiter from the files.
sniffer = csv.Sniffer()

Get the AReM Data Set

the dataset has been downloaded and added to the 'data' directory

In [2]:
# Root Source of the data directory
source_dir = '../data/AReM/'

### (b) Test and Train Data

In [3]:
# Defining the test and train lists to store the file paths
test = []
train = []

# For 
for root, dirs, files in os.walk(source_dir):
    for file_name in files:
        file_path = os.path.join(root, file_name)
        if 'pdf' not in file_name:
            if 'DS_Store' not in file_name:
                file_path = os.path.join(root,file_name)
                if 'bending1' in file_path or 'bending2' in file_path:
                    if 'dataset1.csv' in file_path or 'dataset2.csv' in file_path:
                        test.append(file_path)
                    else:
                        train.append(file_path)
                else:
                    if 'dataset1.csv' in file_path or 'dataset2.csv' in file_path or 'dataset3.csv' in file_path:
                        test.append(file_path)
                    else:
                        train.append(file_path)

In [4]:
#printing the list of files that are going to be in test data
test

['../data/AReM/bending1/dataset1.csv',
 '../data/AReM/bending1/dataset2.csv',
 '../data/AReM/walking/dataset1.csv',
 '../data/AReM/walking/dataset2.csv',
 '../data/AReM/walking/dataset3.csv',
 '../data/AReM/bending2/dataset1.csv',
 '../data/AReM/bending2/dataset2.csv',
 '../data/AReM/standing/dataset1.csv',
 '../data/AReM/standing/dataset2.csv',
 '../data/AReM/standing/dataset3.csv',
 '../data/AReM/sitting/dataset1.csv',
 '../data/AReM/sitting/dataset2.csv',
 '../data/AReM/sitting/dataset3.csv',
 '../data/AReM/lying/dataset1.csv',
 '../data/AReM/lying/dataset2.csv',
 '../data/AReM/lying/dataset3.csv',
 '../data/AReM/cycling/dataset1.csv',
 '../data/AReM/cycling/dataset2.csv',
 '../data/AReM/cycling/dataset3.csv']

In [5]:
#printing the list of files that are going to be in train data
train

['../data/AReM/bending1/dataset7.csv',
 '../data/AReM/bending1/dataset6.csv',
 '../data/AReM/bending1/dataset4.csv',
 '../data/AReM/bending1/dataset5.csv',
 '../data/AReM/bending1/dataset3.csv',
 '../data/AReM/walking/dataset7.csv',
 '../data/AReM/walking/dataset6.csv',
 '../data/AReM/walking/dataset4.csv',
 '../data/AReM/walking/dataset5.csv',
 '../data/AReM/walking/dataset10.csv',
 '../data/AReM/walking/dataset11.csv',
 '../data/AReM/walking/dataset13.csv',
 '../data/AReM/walking/dataset12.csv',
 '../data/AReM/walking/dataset15.csv',
 '../data/AReM/walking/dataset14.csv',
 '../data/AReM/walking/dataset8.csv',
 '../data/AReM/walking/dataset9.csv',
 '../data/AReM/bending2/dataset6.csv',
 '../data/AReM/bending2/dataset4.csv',
 '../data/AReM/bending2/dataset5.csv',
 '../data/AReM/bending2/dataset3.csv',
 '../data/AReM/standing/dataset7.csv',
 '../data/AReM/standing/dataset6.csv',
 '../data/AReM/standing/dataset4.csv',
 '../data/AReM/standing/dataset5.csv',
 '../data/AReM/standing/dataset

### 1(c). Feature Extraction. 
### Classification of time series usually needs extracting features from them. In this problem, we focus on time-domain features.

### 1(c)(i). Research what types of time-domain features are usually used in time series classification and list them (examples are minimum, maximum, mean, etc).

#### i. Research

Some of the time-domain features that are used in the time series classification are as follows

1. Mean
2. Median
3. Standard Deviation
4. Variance
5. Minimum
6. Maximum
7. Range
8. Root Mean Square(RMS)
9. First Quartile
10. Third Quartile
11. Entropy
12. Auto-Correlation
13. Interquartile Range (IQR)
14. Scaling Properties

### 1(c)(ii). Extract the time-domain features minimum, maximum, mean, median, standard deviation, first quartile, and third quartile for all of the 6 time series in each instance. You are free to normalize/standardize features or use them directly.

In [6]:
# Extract the columns from the csv files and add them as per the order they are retrieved
def get_cols():
    feature = []
    STATS = ['mean', 'std', 'min', '1st_quartile', 'median', '3rd_quartile', 'max']
    for i in range(1, 7):
        for stat in STATS:
            feature.append(f'{stat}{i}')
    return feature

def get_feature_list(filelist):
    # Declaring the feature list
    feature = []

    # Order the columns in the csv files
    for i in range(1, 7):
        od_stat = ['min', 'max', 'mean', 'median', 'std', '1st_quartile', '3rd_quartile']
        for stat in od_stat:
            feature.append(f'{stat}{i}')

    # Declare the labels and instances variable
    labels = []
    ins = []
    
    #column order as per the given files
    column_order = ["time","avg_rss12","var_rss12","avg_rss13","var_rss13","avg_rss23","var_rss23"]

    #for each file in the training set list of files, process each of them and extract the data
    for file_path in filelist:
        # Open the file that has been data separated by a delimiter
        file = open(file_path)
    
        file_delimiter = sniffer.sniff(file.read()).delimiter
        #segregate the data extraction method using the different delimiters by dynamically extracting them from the files
        if file_delimiter == ",":
            result_df = pd.read_csv(file_path, skiprows=5, header=None, on_bad_lines='skip')
        elif file_delimiter == "\\s+":
            result_df = pd.read_csv(file_path, skiprows=5, sep="\\s+", header=None, on_bad_lines='skip')
        else:
            result_df = pd.read_csv(file_path, skiprows=5, sep="\\s+", header=None, on_bad_lines='skip')

        #change the column names to the original column names as given in the list below
        column_names_from_act_files = ["# Columns: time","avg_rss12","var_rss12","avg_rss13","var_rss13","avg_rss23","var_rss23"]
        result_df.columns = column_names_from_act_files

        partition=1
        # breaking time series into equal lenghts based on the partiion value above
        partiton_l = int(result_df.shape[0] / partition)
        list1 = []
        for i in range(partition):
            s = i * partiton_l
            e = min((i + 1) * partiton_l, result_df.shape[0])
            cur_partition = result_df[s : e]
            
            # Defining the Time series column name
            time_col = "# Columns: time"
            
            # Removing the time column from the data frame
            cur_list1 = cur_partition.describe().drop('count').drop(columns=time_col).T
            list1.append(cur_list1.values.flatten())
        ins.append(np.concatenate(list1))
        
        #Adding the file labels based on the file path to the dataframe
        labels.append(file_path.split('/')[3])

    #converting the lists to the dataframe
    result_df = pd.DataFrame(ins)
    result_df.columns = get_cols()
    result_df = result_df.loc[:, feature]

    # Adding the labels to the data
    result_df['label'] = pd.Series(labels)

    # Printing the training dataframe information
    return result_df

In [7]:
# Printing the Test Data Frame
test_df = get_feature_list(test)
test_df

,min1,max1,mean1,median1,std1,1st_quartile1,3rd_quartile1,min2,max2,mean2,...,1st_quartile5,3rd_quartile5,min6,max6,mean6,median6,std6,1st_quartile6,3rd_quartile6,label
0,37.25,45.00,40.624792,40.500,1.476967,39.2500,42.0000,0.0,1.30,0.358604,...,33.0000,36.0000,0.0,1.92,0.570583,0.430,0.582915,0.0000,1.300,bending1
1,38.00,45.67,42.812812,42.500,1.435550,42.0000,43.6700,0.0,1.22,0.372437,...,32.0000,34.5000,0.0,3.11,0.571083,0.430,0.601010,0.0000,1.300,bending1
2,19.33,43.50,34.227771,35.500,4.889576,30.5000,37.7500,0.0,14.50,3.995729,...,14.7500,18.6700,0.0,9.74,3.394125,3.100,1.792090,2.1050,4.425,walking
3,12.50,45.00,33.509729,34.125,4.850923,30.5000,36.7500,0.0,13.05,4.450771,...,14.6275,18.7500,0.0,8.96,3.378479,3.085,1.787360,2.0600,4.440,walking
4,15.00,46.75,34.660583,35.000,5.315110,31.0000,38.2500,0.0,13.44,4.200896,...,14.2500,18.5000,0.0,8.99,3.244396,3.000,1.630983,2.1200,4.240,walking
5,12.75,51.00,24.562958,24.250,3.737514,23.1875,26.5000,0.0,6.87,0.590833,...,20.5000,27.0000,0.0,4.97,0.700188,0.500,0.693720,0.4300,0.870,bending2
6,0.00,42.75,27.464604,28.000,3.583582,25.5000,30.0000,0.0,7.76,0.449708,...,15.0000,20.7500,0.0,6.76,1.122125,0.830,1.012342,0.4700,1.300,bending2
7,33.33,48.00,44.334729,45.000,2.476940,42.2500,46.5000,0.0,3.90,0.432958,...,9.3300,17.7500,0.0,5.02,0.933000,0.830,0.673609,0.4700,1.250,standing
8,35.50,46.25,43.174938,43.670,1.989052,42.5000,44.5000,0.0,2.12,0.506583,...,12.7500,16.5000,0.0,5.72,0.911979,0.830,0.666161,0.4700,1.220,standing
9,32.75,47.00,42.760562,44.500,3.398919,41.3300,45.3725,0.0,3.34,0.486167,...,13.0000,18.5650,0.0,5.73,0.842271,0.710,0.722165,0.4300,1.090,standing


In [8]:
# Printing the Train Data Frame
train_df = get_feature_list(train)
train_df

,min1,max1,mean1,median1,std1,1st_quartile1,3rd_quartile1,min2,max2,mean2,...,1st_quartile5,3rd_quartile5,min6,max6,mean6,median6,std6,1st_quartile6,3rd_quartile6,label
0,36.25,48.00,43.969125,44.50,1.618364,43.310,44.67,0.0,1.50,0.413125,...,20.5000,23.7500,0.0,2.96,0.555312,0.490,0.487826,0.0000,0.830,bending1
1,37.00,48.00,43.454958,43.25,1.386098,42.500,45.00,0.0,1.58,0.378083,...,22.2500,24.0000,0.0,5.26,0.679646,0.500,0.622534,0.4300,0.870,bending1
2,33.00,47.75,42.179812,43.50,3.670666,39.150,45.00,0.0,3.00,0.696042,...,30.4575,36.3300,0.0,2.18,0.613521,0.500,0.524317,0.0000,1.000,bending1
3,33.00,45.75,41.678063,41.75,2.243490,41.330,42.75,0.0,2.83,0.535979,...,28.4575,31.2500,0.0,1.79,0.383292,0.430,0.389164,0.0000,0.500,bending1
4,35.00,47.40,43.954500,44.33,1.558835,43.000,45.00,0.0,1.70,0.426250,...,35.3625,36.5000,0.0,1.79,0.493292,0.430,0.513506,0.0000,0.940,bending1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,18.50,44.25,35.752354,36.00,4.614802,33.000,39.33,0.0,12.60,3.328104,...,14.0000,18.0625,0.0,9.39,3.069667,2.770,1.748326,1.7975,4.060,cycling
65,24.25,45.00,37.177042,36.25,3.581301,34.500,40.25,0.0,8.58,2.374208,...,17.9500,21.7500,0.0,9.34,2.921729,2.500,1.852600,1.5000,3.900,cycling
66,23.33,43.50,36.248768,36.75,3.824632,33.415,39.25,0.0,9.71,2.737307,...,15.7500,21.0000,0.0,11.15,3.532463,3.110,1.965267,2.1700,4.625,cycling
67,26.25,44.25,36.957458,36.29,3.434863,34.500,40.25,0.0,8.64,2.420083,...,14.0000,18.2500,0.0,8.34,2.934625,2.525,1.631380,1.6600,4.030,cycling


In [9]:
# Appending the data from both the data frames into a single data frame
combined_df = pd.DataFrame()
combined_df = combined_df.append(train_df)
combined_df = combined_df.append(test_df)
combined_df

,min1,max1,mean1,median1,std1,1st_quartile1,3rd_quartile1,min2,max2,mean2,...,1st_quartile5,3rd_quartile5,min6,max6,mean6,median6,std6,1st_quartile6,3rd_quartile6,label
0,36.25,48.00,43.969125,44.500,1.618364,43.31,44.6700,0.0,1.50,0.413125,...,20.5000,23.7500,0.0,2.96,0.555312,0.49,0.487826,0.0000,0.83,bending1
1,37.00,48.00,43.454958,43.250,1.386098,42.50,45.0000,0.0,1.58,0.378083,...,22.2500,24.0000,0.0,5.26,0.679646,0.50,0.622534,0.4300,0.87,bending1
2,33.00,47.75,42.179812,43.500,3.670666,39.15,45.0000,0.0,3.00,0.696042,...,30.4575,36.3300,0.0,2.18,0.613521,0.50,0.524317,0.0000,1.00,bending1
3,33.00,45.75,41.678063,41.750,2.243490,41.33,42.7500,0.0,2.83,0.535979,...,28.4575,31.2500,0.0,1.79,0.383292,0.43,0.389164,0.0000,0.50,bending1
4,35.00,47.40,43.954500,44.330,1.558835,43.00,45.0000,0.0,1.70,0.426250,...,35.3625,36.5000,0.0,1.79,0.493292,0.43,0.513506,0.0000,0.94,bending1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,24.75,48.33,44.182937,48.000,7.495615,48.00,48.0000,0.0,3.11,0.101875,...,2.0000,5.5425,0.0,3.91,0.692771,0.50,0.675781,0.3225,0.94,lying
15,48.00,48.25,48.004167,48.000,0.032038,48.00,48.0000,0.0,0.43,0.007167,...,4.6700,10.0000,0.0,2.50,0.641229,0.50,0.388372,0.4600,0.83,lying
16,24.25,45.00,37.177042,36.250,3.581301,34.50,40.2500,0.0,8.58,2.374208,...,17.9500,21.7500,0.0,9.34,2.921729,2.50,1.852600,1.5000,3.90,cycling
17,28.75,44.75,37.561187,36.875,3.226507,35.25,40.2500,0.0,9.91,2.080687,...,18.0000,21.5000,0.0,9.62,2.765896,2.45,1.769203,1.4100,3.77,cycling


### 1(c)(iii). Estimate the standard deviation of each of the time-domain features you extracted from the data. Then, use Python’s bootstrapped or any other method to build a 90% bootsrap confidence interval for the standard deviation of each feature.

In [10]:
# Moving all the data from the train_df into a new data_frame for further processing
# Removing the label from the dataframe because it is of no importance here
df = combined_df.iloc[:, :-1]

# Defining the number of bootstrap iterations that shall be performed
n_iterations = 1000

# Defining the confidence level(given in the question to check for the 90% confidence level)
confidence_level = 0.90

# Initializing an empty DataFrame to store bootstrap results after calculations
bootstrap_results = pd.DataFrame(columns=['Time_domain_feat', 'Std_Deviation', 'Low_Bound', 'High_Bound'])

# Looping through each feature available available
for feature in df.columns:
    
    # Initializing an array to store bootstrap sample standard deviations
    bootstrap_std_devs = []
    
    # Performing bootstrapping to estimate the standard deviation based on the number of iterations that have been defined
    for i in range(n_iterations):
        
        # Bootstrap resampling with replacement
        bootstrap_sample = resample(df[feature], replace=True, n_samples=len(df))
        
        # Calculating the standard deviation for the bootstrap sample
        bootstrap_std_dev = np.std(bootstrap_sample)
        
        # Appending the result to the array
        bootstrap_std_devs.append(bootstrap_std_dev)
    
    # Calculating the lower and upper percentiles for the confidence interval
    lower_percentile = (1 - confidence_level) / 2
    upper_percentile = 1 - lower_percentile
    lower_ci = np.percentile(bootstrap_std_devs, lower_percentile * 100)
    upper_ci = np.percentile(bootstrap_std_devs, upper_percentile * 100)
    
    # Appending the results to the bootstrap_results DataFrame
    # Using the ignore_idex as it is a required parameter for appending data to a dictionary
    bootstrap_results = bootstrap_results.append({
        'Time_domain_feat': feature,
        'Std_Deviation': np.std(df[feature]),
        'Low_Bound': lower_ci,
        'High_Bound': upper_ci
    }, ignore_index=True)

# Printing the bootstrap results DataFrame
bootstrap_results

,Time_domain_feat,Std_Deviation,Low_Bound,High_Bound
0,min1,9.515445,8.244868,10.702418
1,max1,4.369322,3.253839,5.324397
2,mean1,5.305297,4.700335,5.820725
3,median1,5.409056,4.763485,5.988436
4,std1,1.762087,1.552211,1.941653
5,1st_quartile1,6.118809,5.516659,6.613733
6,3rd_quartile1,5.109643,4.291035,5.838334
7,min2,0.000000,0.000000,0.000000
8,max2,5.033882,4.598628,5.333324
9,mean2,1.565228,1.385522,1.700068


### 1(c)(iv). Use your judgement to select the three most important time-domain features (one option may be min, mean, and max)

Selecting the three most important time-domain features from a set of features extracted from time series data - 

1. Mean
2. Standard Deviation
3. Maximum or Minimum

## 2. ISLR 3.7.4

I collect a set of data (n = 100 observations) containing a single predictor and a quantitative response. I then fit a linear regression model to the data, as well as a separate cubic regression, i.e. Y = β0 +β1X +β2X2 +β3X3 +ε.

### (a) Linear Train

Since in a lot of regression predictors, the degree of the regression is directly proportional to the fitting of the model. Hence, <b>the RSS for the cubic(polynomial) regression shall be lesser than the RSS for the linear regression</b>. Because, the extra cubic(polynomial) terms would allow a greater fit for the training data.

### (b) Linear Test

Converse to the above answer(a). So, in this case I would expect the <b>Test RSS of the cubic(polynomial) regression to be higher than the RSS of the Linear regression</b>. The cubic model is likely to overfit on the training data and cause more error than the linear model.

### (c) Not Linear Train

Here, <b>the cubic(polynomial) regression will have a lower train RSS than that of the Linear regression model</b> as the cubic regression model can better fit the non linear data than the linear regression model

### (d) Not Linear Testing

<b> The Test RSS of the Cubic(polynomial) regression model will be lesser than the Test RSS of the linear regression model </b>